In [1]:
!pip install --force-reinstall scikit-surprise

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached joblib-1.5.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached scipy-1.15.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached joblib-1.5.0-py3-none-any.whl (307 kB)
Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl (12.6 MB)
Using cached scipy-1.15.3-cp312-cp312-win_amd64.whl (41.0 MB)
Failed to build scikit-surprise


  error: subprocess-exited-with-error
  
  × Building wheel for scikit-surprise (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [155 lines of output]
      C:\Users\DELL\AppData\Local\Temp\pip-build-env-0f0lbs9q\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              *****************************************

In [2]:
!pip install scikit-surprise


In [3]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy

In [4]:
df = pd.read_csv('fashion_products.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   User ID       1000 non-null   int64  
 1   Product ID    1000 non-null   int64  
 2   Product Name  1000 non-null   object 
 3   Brand         1000 non-null   object 
 4   Category      1000 non-null   object 
 5   Price         1000 non-null   int64  
 6   Rating        1000 non-null   float64
 7   Color         1000 non-null   object 
 8   Size          1000 non-null   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 70.4+ KB


In [6]:
df.sample(10)

,User ID,Product ID,Product Name,Brand,Category,Price,Rating,Color,Size
614,44,615,T-shirt,Nike,Kids' Fashion,67,2.894450,Red,L
959,94,960,Jeans,Gucci,Men's Fashion,50,4.139661,Red,M
309,61,310,Jeans,H&M,Men's Fashion,83,3.945481,White,XL
440,47,441,Sweater,Adidas,Men's Fashion,50,1.841606,Black,XL
480,52,481,Jeans,Zara,Women's Fashion,24,1.671659,Blue,XL
202,53,203,Jeans,Zara,Women's Fashion,75,1.876205,Yellow,S
778,98,779,Jeans,Nike,Kids' Fashion,94,3.726036,Yellow,XL
25,26,26,Dress,Gucci,Women's Fashion,21,4.748169,Green,S
936,80,937,T-shirt,Gucci,Men's Fashion,38,1.123240,Red,S
181,39,182,Jeans,Zara,Men's Fashion,68,4.358913,Yellow,L


In [7]:
df.head()

,User ID,Product ID,Product Name,Brand,Category,Price,Rating,Color,Size
0,19,1,Dress,Adidas,Men's Fashion,40,1.043159,Black,XL
1,97,2,Shoes,H&M,Women's Fashion,82,4.026416,Black,L
2,25,3,Dress,Adidas,Women's Fashion,44,3.337938,Yellow,XL
3,57,4,Shoes,Zara,Men's Fashion,23,1.049523,White,S
4,79,5,T-shirt,Adidas,Men's Fashion,79,4.302773,Black,M


In [8]:
df.tail()

,User ID,Product ID,Product Name,Brand,Category,Price,Rating,Color,Size
995,20,996,Shoes,Zara,Women's Fashion,55,1.620081,Black,M
996,42,997,Sweater,Nike,Kids' Fashion,13,1.544464,Green,L
997,9,998,Sweater,Zara,Men's Fashion,47,3.961913,White,L
998,8,999,T-shirt,Zara,Women's Fashion,68,3.089722,Blue,S
999,91,1000,Shoes,Adidas,Women's Fashion,79,3.093553,White,L


In [9]:
df = pd.DataFrame(df, columns=['User ID', 'Product ID', 'Product Number', 'Product Name', 
                                'Brand', 'Category', 'Price', 'Rating', 'Color', 'Size'])

In [10]:
# Check for missing values
print("Missing values in the dataset:")
print(df.isnull().sum())


Missing values in the dataset:
User ID              0
Product ID           0
Product Number    1000
Product Name         0
Brand                0
Category             0
Price                0
Rating               0
Color                0
Size                 0
dtype: int64


In [11]:
# Basic stats about the ratings
print("\nRating statistics:")
print(df['Rating'].describe())


Rating statistics:
count    1000.000000
mean        2.993135
std         1.153185
min         1.000967
25%         1.992786
50%         2.984003
75%         3.985084
max         4.987964
Name: Rating, dtype: float64


In [19]:
# We'll use 'User ID', 'Product ID', and 'Rating'
reader = Reader(rating_scale=(df['Rating'].min(), df['Rating'].max()))
data = Dataset.load_from_df(df[['User ID', 'Product ID', 'Rating']], reader)


In [20]:

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [21]:
algo = SVD()
algo.fit(trainset)


In [22]:
predictions = algo.test(testset)
print("RMSE:", accuracy.rmse(predictions))
print("MAE:", accuracy.mae(predictions))


RMSE: 1.1258
RMSE: 1.1258094907887664
MAE:  0.9806
MAE: 0.9806204712111523


In [23]:
user_id = 19
all_product_ids = df['Product ID'].unique()
# Get products the user has already rated
rated_products = df[df['User ID'] == user_id]['Product ID'].values
# Recommend products not yet rated
products_to_predict = [pid for pid in all_product_ids if pid not in rated_products]

predictions = []
for pid in products_to_predict:
    pred = algo.predict(user_id, pid)
    predictions.append((pid, pred.est))

In [24]:
# Sort by predicted rating
top_recs = sorted(predictions, key=lambda x: x[1], reverse=True)[:5]
print("Top 5 product recommendations for user", user_id)
for pid, est in top_recs:
    prod_name = df[df['Product ID'] == pid]['Product Name'].iloc[0]
    print(f"Product ID: {pid}, Name: {prod_name}, Predicted Rating: {est:.2f}")

Top 5 product recommendations for user 19
Product ID: 534, Name: Dress, Predicted Rating: 3.05
Product ID: 939, Name: Jeans, Predicted Rating: 3.04
Product ID: 93, Name: Shoes, Predicted Rating: 3.02
Product ID: 704, Name: T-shirt, Predicted Rating: 3.01
Product ID: 616, Name: Dress, Predicted Rating: 3.00


In [25]:
def recommend_products(user_id, n=5):
    all_product_ids = df['Product ID'].unique()
    rated_products = df[df['User ID'] == user_id]['Product ID'].values
    products_to_predict = [pid for pid in all_product_ids if pid not in rated_products]
    predictions = [ (pid, algo.predict(user_id, pid).est) for pid in products_to_predict ]
    top_recs = sorted(predictions, key=lambda x: x[1], reverse=True)[:n]
    for pid, est in top_recs:
        prod_name = df[df['Product ID'] == pid]['Product Name'].iloc[0]
        print(f"Product ID: {pid}, Name: {prod_name}, Predicted Rating: {est:.2f}")

# Usage
recommend_products(19)


Product ID: 534, Name: Dress, Predicted Rating: 3.05
Product ID: 939, Name: Jeans, Predicted Rating: 3.04
Product ID: 93, Name: Shoes, Predicted Rating: 3.02
Product ID: 704, Name: T-shirt, Predicted Rating: 3.01
Product ID: 616, Name: Dress, Predicted Rating: 3.00
